# **Intro to Speech Processing:**

*Yadong Liu & Arian Shamei*

**Goal:**  Understand how speech data is discussed, measured, processed, and implemented into deep learning architectures

The following tutorial will make use to the Alcohol Language Corpus (Schiel & Barfusser), which contains the sober and intoxicated speech from over 100 individuals. The tutorial will cover the following steps for pre-processing speech data using Parselmouth, Librosa, Python_Speech_Features and other relevant packages. 

Preprocessing code obtained from the docs page of parselmouth, librosa, and python speech features.
Model and dataset building code obtained from Seth Adams: https://github.com/seth814/Audio-Classification

**Targets:**
1. Evaluating specific metrics from an audio file. 
2. Pre-emphasis of the audio-file for acoustic feature augmentation
3. Generation of Waveforms, Spectrograms, Mel spectrograms, Mel Formant Cepstral Coefficients
4. Segmentation of audio data to manageable sizes.
5. Incorporate speech data in a deep learning task

In [ ]:
import pandas as pd
import librosa
from librosa import display
from matplotlib import cm
import pydub
from pydub.audio_segment import AudioSegment
from pydub.utils import make_chunks
import os
import matplotlib.pyplot as plt
import numpy as np
from librosa.effects import trim
import scipy
from scipy.io.wavfile import read
import python_speech_features
from python_speech_features import mfcc
from python_speech_features import logfbank
from tqdm import tqdm
import scipy.io.wavfile as wav
import parselmouth
import keras 
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,LSTM, MaxPooling2D, LocallyConnected2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras import optimizers
from keras.callbacks import ModelCheckpoint
directory = "C:/Users/User/Desktop/all.ALC.4.cmdi.44539.1573493756/ALC/"
path = "C:/Users/User/Desktop/clean/"
df = pd.read_table(r"C:\Users\User\Desktop\IS2011CHALLENGE\TRAIN.tbl", encoding = "utf8")
df.set_index('file', inplace=True)

1. **What is speech?**

We begin this tutorial with a brief overview of speech itself, including how speech is described, measured, discussed. 

    Signals: 
    Traditionally, phoneticians view speech as an acoustic signal consisting of a complex wave composed of many individual sine waves; the speaker's movements drive air molecules via compression and rarefaction, this in turn propogates energy to other air molecules which eventually influence movement along a sensitive membrane inside the head of surrounding humans. This concept is known as the speech chain. Crucial to the speech chain is the presence of a human listener.

    Properties of the air molecules' movement determine how it is perceived. The rate at which an air molecule cycles during compression and rarefaction is referred to as the frequency of a sound, and the degree of pressure underlying the movement is referred to as the amplitude. Frequency is perceived by the listener as pitch, and amplitude is perceived as loudness (intensity). Each sound in speech is a modulation of frequencies and amplitude of the surrounding air, which combined with a time signal, allow for a system of communication.

Praat is a highly popular acoustic analysis software which allows sophisticated measurements and analysis of audio signal. Advanced usage of Praat requires familiarity with PraatScript, but parselmouth is a great python interface which allows access to Praat. 


**Using parselmouth, lets take measurements of a speech along the dimensions we've discussed. Starting by generating a waveform of the first sound file in the dataset.**

In [ ]:
snd = parselmouth.Sound(directory + df.index[0][8:])
print(np.shape(snd))
plt.figure()
plt.plot(snd.xs(), snd.values.T)
plt.xlim([snd.xmin, snd.xmax])
plt.xlabel("time [s]")
plt.ylabel("amplitude")
plt.show()

A waveform is a direct representation of the signal, with amplitude on the Y-axis and time on the X-axis.

When np.shape() is called on the sound file, we get (1, 448938).The second value is the total number of samples in the audio file.

Note that the audio file is just over 10 seconds long. What does this tell us about the sampling rate?
This likely means we are sampling 44,100 times per second.


**Now lets calculate the speaker's mean pitch from the audio file using Parselmouth's to_pitch function.**

In [ ]:

pitch = snd.to_pitch()
pitch_values = [f for f in pitch.selected_array['frequency'] if f > 0 and f < 200]
np.asarray(pitch_values).mean()

**Now calculate average intensity across the soundfile**:

In [ ]:
intensity = snd.to_intensity()
np.asarray(intensity).mean()


The waveform we generated earlier is a visualization with amplitude on the y axis and time on the x axis. Most acoustic analysis is done using *spectrograms*, a three dimensional visualization with frequency on the y axis, time on the x axis, and amplitude represented by darkness.

Unlike the waveform above, spectrograms make use of *Fourier transforms* to decompose the signal into its component frequencies.

The code below will plot spectograms generated by parselmouth, and also plot pitch and amplitude contours using the measurements we calculated earlier

In [ ]:
def draw_spectrogram(spectrogram, dynamic_range=70):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
    plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap='afmhot')
    plt.ylim([spectrogram.ymin, spectrogram.ymax])
    plt.xlabel("time [s]")
    plt.ylabel("frequency [Hz]")
    
def draw_pitch(pitch):
    # Extract selected pitch contour, and
    # replace unvoiced samples by NaN to not plot
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=5, color='w')
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=2)
    plt.grid(False)
    plt.ylim(0, pitch.ceiling)
    plt.ylabel("fundamental frequency [Hz]")
    
def draw_intensity(intensity):
    plt.plot(intensity.xs(), intensity.values.T, linewidth=3, color='w')
    plt.plot(intensity.xs(), intensity.values.T, linewidth=1)
    plt.grid(False)
    plt.ylim(0)
    plt.ylabel("intensity [dB]")


Now lets generate a spectrogram in Parselmouth, and plot it with the amplitude contour outlined

In [ ]:
intensity = snd.to_intensity()
spectrogram = snd.to_spectrogram()
plt.figure()
draw_spectrogram(spectrogram)
plt.twinx()
draw_intensity(intensity)
plt.xlim([snd.xmin, snd.xmax])
plt.show()




Note that the spectrogram above is somewhat muddy and difficult to interpret. This is known as a wide-band spectrogram. Wide-band spectrograms have better resolution for time than frequency. We can generate a narrow band spectrogram instead by increasing the window length.

We can also add pre-emphasis to the audio to enhance frequencies used in speech. The code below plots a pre-emphasized narrow-band spectrogram, and outlines the pitch contour.


In [ ]:
pre_emphasized_snd = snd.copy()
pre_emphasized_snd.pre_emphasize()
spectrogram = pre_emphasized_snd.to_spectrogram(window_length=0.03, maximum_frequency=8000)
plt.figure()
draw_spectrogram(spectrogram)
plt.twinx()
draw_pitch(pitch)
plt.xlim([snd.xmin, snd.xmax])
plt.show()

Note that pre-emphasis enhanced the frequencies used in speech, while ignoring the non-speech noise at the very end of the recording. 

The audio we have been plotting is ten seconds long. Using parselmouth, we can extract any specific portion of the audio file.

In [ ]:
# Create a snippet of the recording
snd_part = snd.extract_part(from_time=1.0,to_time=3.0, preserve_times=False)
pre_emphasized_snd = snd_part.copy()
#pre-emphasize the snippet
pre_emphasized_snd.pre_emphasize()
#plot pre-emphasized snippet spectrogram
spectrogram = pre_emphasized_snd.to_spectrogram(window_length=0.03, maximum_frequency=8000)
plt.figure()
draw_spectrogram(spectrogram)
plt.xlim([snd_part.xmin, snd_part.xmax])
plt.show()

In the spectrogram above, the bright horizontal lines are *formants* which are the spectral components of vowels. 

The grainy bars of *broad-spectrum noise* which appear between formants are the spectral components of consonants. 

Another variation of spectrogram is the *mel spectrogram*. Mel spectrograms constrain spectrograms along the *mel scale*. 
The mel scale is a non-linear transformation of a frequency range to represent it according to human perception.

Humans are very sensitive to different frequencies in the lower ranges (100Hz vs 300 Hz), but this difference is lost at greater values (3000Hz vs 3300Hz). The mel scale accurately represents this change in resolution. Incorporating the mel scale into deep learning models assists them in learning human concepts. 


Using another audio processing toolkit, *Librosa*, let's generate a mel spectrogram for the same audio file.

In [ ]:
y, sr = librosa.load(directory + df.index[0][8:], sr = 44100)
S = librosa.feature.melspectrogram(y, sr=sr, n_mels=256, n_fft = 2048, hop_length=32,
                                   fmax=8000)
S_dB = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_dB,sr=sr,
                             fmax=8000)


Librosa also has a handry trim function, which will remove leading and trailing silence audio files. 

Lets use this trim function on the dataset to clean the audio files in preparation for our model. Simultaneously, to speed up data processing, we are going to downsample the data from 44.1KHz to 22KHz. The cleaned training files can be found in the 'clean' folder, and the dev/test set files can be found in the 'cleantest' folder

In [ ]:
y, sr = librosa.load(directory + df.index[0][8:])
yt,index = librosa.effects.trim(y, top_db=25)
S = librosa.feature.melspectrogram(y, sr=22050, n_mels=256, n_fft = 2048, hop_length=32,
                                   fmax=8000)
S_dB = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_dB,sr=sr,
                             fmax=8000)

Another type of mel-based visualization widely employed in audio DL models is the Mel Frequency Cepstral Coefficient (MFCC):
MFCCs are a series of short-term power spectrums over time, where frequency components are sorted into mel-banks on the y-axis across time on the x-axis. 

Both Librosa and python_speech_features allows very easy MFCC generation. We will use python_speech_features to plot an MFCC across one second of audio from a file in our dataset. 

In [ ]:
rate, sig = scipy.io.wavfile.read(directory+df.index[1][8:])
mfcc_feat = mfcc(sig[rate*2:rate*3],rate, numcep=13, nfilt=26, nfft=1103)

ig, ax = plt.subplots()
mfcc_data= np.swapaxes(mfcc_feat, 0 ,1)
cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.hot, origin='lower', aspect='auto')
ax.set_title('MFCC')
#Showing mfcc_data
plt.show()


We are going to train our model using MFCCs generated over one second of audio. To generate the correct amount of samples, we are going to need to know the length of each file, and the distribution of data over each condition

In [ ]:
for f in df.index:
    rate, signal = scipy.io.wavfile.read(path+f[16:])
    df.at[f, 'length'] = signal.shape[0]/rate
df

In [165]:
classes = list(np.unique(df.state))
class_dist = df.groupby(['state'])['length'].mean()
class_sum = df.groupby(['state'])['length'].sum()
class_avg_len = df.groupby(['state'])['length'].mean()
prob_dist = class_dist / class_dist.sum()
n_samples = 2*int(df['length'].sum())

In [164]:
prob_dist = class_dist / class_dist.sum()


state
I    0.344025
S    0.655975
Name: length, dtype: float64

In [166]:
class_dist

state
I    10.021556
S     9.662736
Name: length, dtype: float64

In [ ]:
class_avg_len

Now that we know the length of our files, we know how many one second samples to take from the data. 

Below is a function that will randomly sample one second MFCCs from audio files to build our training set. 

In [ ]:
def build_rand_feat():
    X=[]
    y=[]
    #for data normalization we will create a rolling log of minimum and maximum values
    _min, _max = float('inf'), -float('inf')
    for _ in tqdm(range(n_samples)):
        #sample from each class randomly according to the calculated probability distribution:
        rand_class= np.random.choice(class_dist.index, p=prob_dist)
        file = np.random.choice(df.index)
        rate, wav  = scipy.io.wavfile.read(path+file[16:])
        label = df.at[file, 'state']
        #Ensures that each file is long enough for a one second sample (at least 2 seconds long)
        if wav.shape[0] > rate*2:
            #choose a random timepoint in the audio file to draw the one-second sample
            rand_index = np.random.randint(0, wav.shape[0]-config.step)
            sample = wav[rand_index:rand_index+config.step]
            #generate MFCC from the random sample
            X_sample = mfcc(sample, rate, 
                          numcep=config.nfeat, 
                          nfilt =  config.nfilt,
                          nfft = config.nfft).T
            #update rolling normalization window
            _min = min(np.amin(X_sample), _min)
            _max = max(np.amax(X_sample), _max)
            #append MFCC vector
            X.append(X_sample if config.mode == 'conv' else X_sample.T)           
            y.append(classes.index(label))
        else:
            continue
    X, y = np.array(X), np.array(y)
    X = (X - _min) / (_max - _min)
    if config.mode == 'conv':
        X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1 )
    y= to_categorical(y, num_classes=2)
    return X, y


Now that we have built our random feature set for the audio data, we define our model. As a starting poiny, we have chosen a two layer 2DCNN, with pooling at each layer and  dropout on the final layer. The optimizer adadelta was used as in past experiments it worked well for speaker independent data. 


In [ ]:
def get_conv_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape,use_bias=True,kernel_initializer='normal', padding ="same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())  #

    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))

    adadelta = optimizers.adadelta()
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer=adadelta,
              metrics=['accuracy'])
    return model

In [ ]:
class Config:
    def __init__(self, mode='conv', nfilt=26, nfeat=13, nfft=2048, rate=22050, n_mels=256, hoplength=32,):
        self.mode = mode
        self.nfilt = nfilt
        self.nfeat = nfeat
        self.rate = rate
        self.nfft= nfft
        self.step = int(rate)
        self.nmels = n_mels
        self.hoplength = hoplength
    


In [ ]:
config = Config(mode='conv')
if config.mode == 'conv':
    X,y = build_rand_feat()
    y_flat = np.argmax(y, axis=1)
    input_shape = (X.shape[1], X.shape[2],1)
    model = get_conv_model()

In [18]:
model.fit(X, y, epochs=100, batch_size =32, 
         shuffle=True,validation_data=(X_test,y_test), verbose=2)

Train on 90932 samples, validate on 49628 samples
Epoch 1/100
 - 52s - loss: 0.6444 - acc: 0.6527 - val_loss: 0.7829 - val_acc: 0.4969
Epoch 2/100
 - 39s - loss: 0.6337 - acc: 0.6574 - val_loss: 0.7371 - val_acc: 0.5111
Epoch 3/100
 - 39s - loss: 0.6275 - acc: 0.6644 - val_loss: 0.7263 - val_acc: 0.5152
Epoch 4/100
 - 39s - loss: 0.6239 - acc: 0.6679 - val_loss: 0.7426 - val_acc: 0.5152
Epoch 5/100
 - 39s - loss: 0.6213 - acc: 0.6693 - val_loss: 0.7481 - val_acc: 0.5215
Epoch 6/100
 - 39s - loss: 0.6170 - acc: 0.6731 - val_loss: 0.7440 - val_acc: 0.5325
Epoch 7/100
 - 39s - loss: 0.6132 - acc: 0.6750 - val_loss: 0.7625 - val_acc: 0.5241
Epoch 8/100
 - 39s - loss: 0.6100 - acc: 0.6772 - val_loss: 0.7524 - val_acc: 0.5347
Epoch 9/100
 - 39s - loss: 0.6082 - acc: 0.6799 - val_loss: 0.7414 - val_acc: 0.5404
Epoch 10/100
 - 39s - loss: 0.6058 - acc: 0.6808 - val_loss: 0.7719 - val_acc: 0.5275
Epoch 11/100
 - 39s - loss: 0.6034 - acc: 0.6815 - val_loss: 0.7702 - val_acc: 0.5339
Epoch 12/100


In [ ]:
def build_test2_feat():
    X_test2=[]
    y_test2=[]
    _min, _max = float('inf'), -float('inf')
    for _ in tqdm(range(t2_samples)):
        rand_class= np.random.choice(t2_class_dist.index, p=t2_prob_dist)
        file = np.random.choice(df3.index)
        rate, wav  = scipy.io.wavfile.read(directory+file[16:])
        label = df3.at[file, 'state']
        if wav.shape[0] > rate*2:
            rand_index = np.random.randint(0, wav.shape[0]-config.step)
            sample = wav[rand_index:rand_index+config.step]
            X_sample = mfcc(sample, rate, 
                          numcep=config.nfeat, 
                          nfilt =  config.nfilt,
                          nfft = config.nfft).T
            _min = min(np.amin(X_sample), _min)
            _max = max(np.amax(X_sample), _max)
            X_test2.append(X_sample if config.mode == 'conv' else X_sample.T)           
            y_test2.append(classes.index(label))
        else:
            continue
    X_test2, y_test2 = np.array(X_test2), np.array(y_test2)
    X_test2 = (X_test2 - _min) / (_max - _min)
    if config.mode == 'conv':
        X_test2 = X_test2.reshape(X_test2.shape[0], X_test2.shape[1], X_test2.shape[2], 1 )
    elif config.mode== 'time':
        X_test2 = X_test2.reshape(X_test2.shape[0], X_test2.shape[1], X_test2.shape[2] )
    y_test2= to_categorical(y_test2, num_classes=2)
    return X_test2, y_test2